In [12]:
import os

import numpy as np

from photutils.utils import ImageDepth
from photutils.segmentation import SourceFinder

from astropy.io import fits
from astropy.stats import SigmaClip

import matplotlib.pyplot as plt

In [13]:
def measure():

    '''
    This function calculates the depth of the F275W observations of the Sunburst Arc
    '''

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data'
    results = f'{home}/results'
    
    # Set the file path to the F275W observation
    f275w_file = f'{data}/hst/V5.0_PSZ1G311.65-18.48_F275W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits'

    # Get the inverse sensitivity of the image from the FITS header of the file
    inverse_sensitivity = fits.getheader(f275w_file)['PHOTFLAM']

    # Get the data array of the F275W observation for a core area of the drizzled image, where all the
    # frames commonly overlap. We want to calculate the depth of the image from this portion of the image
    # because this is where the Sunburst Arc also lies, so it is the best representation of the depth
    # achieved on the Sunburst Arc
    f275w = fits.getdata(f275w_file)[3500:5000, 3500:5500]

    # Convert the data from counts/s to erg/s/cm^2/Å
    f275w = f275w * inverse_sensitivity

    # Convert the data to a frequency-space flux density (erg/s/cm^2/Hz) based on the pivot wavelength of
    # the WFC3/UVIS F275W filter
    f275w = f275w * 2709.7**2 / (3e8 * 1e10)

    # Create a mask representing the location of any NaNs in the image
    nan_mask = np.isnan(f275w)

    # Create a mask of the sources in the image
    finder = SourceFinder(npixels=9, progress_bar=False)
    mask = finder(np.where(nan_mask, 0, f275w), abs(np.nanmedian(f275w))).make_source_mask()

    #for i in range(1,11):
    # Determine the depth of the image at a significance of 5σ, using AB magnitudes. Use apertures with radius corresponding
    # to the approximate FWHM of the image PSF. Since the apertures use the summed flux densities within the aperture rather
    # than the average or typical flux density, there is a strong dependence between the estimated depth of the image and the
    # size of the apertures. I think it is most appropriate to set the aperture size to the effective size of an idealized 
    # point source in the image to represent the faintest point source the image could detect
    depth = ImageDepth(1.25, nsigma=5., mask_pad=2., napers=1000, niters=1000, seed=0, zeropoint=-48.6, progress_bar=False)
    limits = depth(f275w, (nan_mask | mask))

    #np.savetxt(f'{results}/flux_limit_measurements.txt', np.stack(depth.fluxes), delimiter=' ', header=' ', comments='#', encoding='utf-8')

In [14]:
measure()